In [2]:
from classiq import (
    PHASE,
    QArray,
    QBit,
    CInt,
    Output,
    X,
    allocate,
    control,
    create_model,
    invert,
    qft,
    repeat,
    show,
    synthesize,
    write_qmod,
)
from classiq.qmod import *
from classiq.qmod.symbolic import pi

@qfunc
def phase_rot(x: QArray[QBit], N: CInt) -> None:
    repeat(
        count=x.len,
        iteration=lambda i: repeat(
            count=x.len,
            iteration=lambda j: repeat(
                count=x.len,
                iteration=lambda k: PHASE(
                    (2 * pi * (2 ** (i + j + k))) / N, x[i]
                ),
            ),
        ),
    )

@qfunc
def ccmod_add(
    N: CInt,
    y: QArray[QBit],
    c1: QBit,
    c2: QBit,
    aux: QBit
) -> None:
    ctrl = QArray("ctrl")
    bind([c1, c2], ctrl)
    control(ctrl, lambda: phase_rot(y, N))
    bind(ctrl, [c1, c2])

@qfunc
def main(
    b: Output[QNum],
    c: Output[QArray[8]],
    ctrl: Output[QArray[2]],
    aux: Output[QBit]
) -> None:
    allocate(8, c)
    allocate(1, b)
    allocate(2, ctrl)
    allocate(1, aux)

    inplace_prepare_int(8, b)
    X(ctrl[0])
    X(ctrl[1])

    within_apply(
        lambda: qft(b),
        lambda: ccmod_add(8, c, ctrl[0], ctrl[1], aux),
    )

qmod = create_model(main)



qprog = synthesize(qmod)
write_qmod(qmod, "ccmod_add_model")
show(qprog)


Opening: https://platform.classiq.io/circuit/6bb24848-33f9-42e9-bd0b-aea20954473f?version=0.44.0


In [3]:
from classiq import execute

results = execute(qprog).result()
res = results[0].value

In [4]:
res

ExecutionDetails(vendor_format_result={}, counts={'011000000000': 2048}, counts_lsb_right=True, parsed_states={'011000000000': {'b': 0.0, 'c': [0, 0, 0, 0, 0, 0, 0, 0], 'ctrl': [1, 1], 'aux': 0.0}}, histogram=None, output_qubits_map={'b': (0,), 'c': (1, 2, 3, 4, 5, 6, 7, 8), 'ctrl': (9, 10), 'aux': (11,)}, state_vector=None, parsed_state_vector_states=None, physical_qubits_map={'b': (8,), 'c': (0, 1, 2, 3, 4, 5, 6, 7), 'ctrl': (9, 10), 'aux': (11,)}, num_shots=2048)